Importing Libraries.

In [53]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,SGD,RMSprop

Loading and preprocessing data

In [57]:
# Data Augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=30,  # Increase rotation range
    width_shift_range=0.3,  # Increase horizontal shift
    height_shift_range=0.3,  # Increase vertical shift
    shear_range=0.3,  # Add shear transformations
    zoom_range=0.3,  # Increase zoom
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data Augmentation for validation set (only rescaling)
val_datagen = ImageDataGenerator(rescale=1./255)

# Set the directories for training and validation data
train_dir = '/content/images/train'
val_dir = '/content/images/validation'

# Flow training images in batches of 64 using train_datagen
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=128,
    color_mode="grayscale",
    class_mode='categorical',
    shuffle=True
)

# Flow validation images in batches of 64 using val_datagen
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    batch_size=128,
    color_mode="grayscale",
    class_mode='categorical',
    shuffle=False
)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


Building the CNN Model

In [38]:
model = Sequential()

# Input Layer
model.add(Input(shape=(48, 48, 1)))

# Convolutional Layer 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolutional Layer 2
model.add(Conv2D(128, (5, 5), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolutional Layer 3
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolutional Layer 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten the output
model.add(Flatten())

# Fully Connected Layer 1
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

# Fully Connected Layer 1
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))


Compiling the model.

In [39]:
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Early stopping to avoid overfitting

In [46]:
early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=5,
                          verbose=1,
                          restore_best_weights=True
                          )

Learning rate scheduler

In [54]:
reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=5,
                              verbose=1,
                              min_delta=0.0001)

Training the model

In [58]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping, reduce_learningrate]
)

Epoch 1/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 28s 119ms/step - accuracy: 0.5760 - loss: 1.1172 - val_accuracy: 0.6378 - val_loss: 0.9685 - learning_rate: 3.2000e-07
Epoch 2/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 27s 117ms/step - accuracy: 0.5705 - loss: 1.1348 - val_accuracy: 0.6381 - val_loss: 0.9683 - learning_rate: 3.2000e-07
Epoch 3/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 40s 113ms/step - accuracy: 0.5653 - loss: 1.1372 - val_accuracy: 0.6376 - val_loss: 0.9684 - learning_rate: 3.2000e-07
Epoch 4/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 42s 115ms/step - accuracy: 0.5702 - loss: 1.1297 - val_accuracy: 0.6381 - val_loss: 0.9681 - learning_rate: 3.2000e-07
Epoch 5/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 27s 116ms/step - accuracy: 0.5682 - loss: 1.1294 - val_accuracy: 0.6376 - val_loss: 0.9696 - learning_rate: 3.2000e-07
Epoch 6/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 27s 115ms/step - accuracy: 0.5698 - loss: 1.1272 - val_accuracy: 0.6374 - val_loss: 0.9690 - learning_rate: 3.2000e-07
Epoch 7/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 26s 11

Evaluating the model

In [59]:
test_loss, test_accuracy = model.evaluate(val_generator)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Loss: {test_loss:.4f}')

56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.5718 - loss: 1.0957
Test Accuracy: 63.81%
Test Loss: 0.9681


Saving the model

In [51]:
model.save('model.keras')